In [1]:
import pandas as pd

#민형기 교수님이 추가된 파일
file_path = '/Users/2suyeon/Desktop/zerobase/미니프로젝트/ML_Project/youquiz_Ml_project/data/final_0214_min.csv'
data = pd.read_csv(file_path)


data.head(), data.describe(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  837 non-null    int64  
 1   출연자         837 non-null    object 
 2   회차          837 non-null    int64  
 3   구독자수        837 non-null    int64  
 4   시청률         835 non-null    float64
 5   직업          837 non-null    object 
 6   성별          837 non-null    object 
 7   나이          837 non-null    object 
 8   수상여부        837 non-null    object 
 9   인지도         837 non-null    object 
 10  년           837 non-null    int64  
 11  월           837 non-null    int64  
 12  일           837 non-null    int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 85.1+ KB


(   Unnamed: 0  출연자  회차  구독자수  시청률   직업 성별  나이 수상여부 인지도     년   월   일
 0           0  나영석  40     0  2.6  서비스  M  중년    Y   Y  2019  10  22
 1           1  이명진  40     0  2.6  서비스  M  청년    N   N  2019  10  22
 2           2  김부연  41     0  2.7   기타  F  중년    N   N  2019  10  29
 3           3  김유자  41     0  2.7   기타  F  청년    N   N  2019  10  29
 4           4  김만순  41     0  2.7   기타  F  노년    N   N  2019  10  29,
        Unnamed: 0          회차           구독자수         시청률            년  \
 count  837.000000  837.000000     837.000000  835.000000   837.000000   
 mean   417.998805  132.351254  555386.308244    4.335001  2021.434886   
 std    241.763316   55.538455  277143.966024    1.064374     1.291165   
 min      0.000000   40.000000       0.000000    2.009000  2019.000000   
 25%    209.000000   85.000000  480000.000000    3.659000  2020.000000   
 50%    418.000000  132.000000  629519.000000    4.422000  2021.000000   
 75%    627.000000  178.000000  764019.000000    4.988000  20

In [2]:
# 민형기 교수님 확인 # 회차에 따른 년 월 일 지정 시청률 null
min_hyung_ki_data = data[data['출연자'] == '민형기']

min_hyung_ki_data


,Unnamed: 0,출연자,회차,구독자수,시청률,직업,성별,나이,수상여부,인지도,년,월,일
835,835,민형기,230,897000,NaN,전문기술,M,중년,Y,Y,2024,2,7
836,835,민형기,230,897000,NaN,전문기술,M,중년,Y,N,2024,2,7


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Remove rows with missing target (시청률)
data_clean = data.dropna(subset=['시청률'])

# Selecting features and target variable
features = ['구독자수', '직업', '성별', '나이', '수상여부', '인지도']
target = '시청률'

X = data_clean[features]
y = data_clean[target]

# Encoding categorical variables
categorical_features = ['직업', '성별', '나이', '수상여부', '인지도']
numerical_features = ['구독자수']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Splitting the dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
model = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100, random_state=42))

# Train the model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

mse


0.2544575758140972

In [4]:
# Preparing '민형기' data for prediction
min_hyung_ki_features = min_hyung_ki_data[features].iloc[0].to_frame().T  # Using the first row as representative

# Predict '민형기's viewership rating
min_hyung_ki_predicted_rating = model.predict(min_hyung_ki_features)

min_hyung_ki_predicted_rating


array([5.19004])

### 날짜 특성에 sin과 cos인 값으로 변경


In [5]:
import numpy as np

# Function to convert dates to sin/cos values
def date_to_sin_cos(df, column_name):
    max_value = df[column_name].max()
    sin_col = np.sin(2 * np.pi * df[column_name] / max_value)
    cos_col = np.cos(2 * np.pi * df[column_name] / max_value)
    return sin_col, cos_col

# Applying the conversion
data_clean['년_sin'], data_clean['년_cos'] = date_to_sin_cos(data_clean, '년')
data_clean['월_sin'], data_clean['월_cos'] = date_to_sin_cos(data_clean, '월')
data_clean['일_sin'], data_clean['일_cos'] = date_to_sin_cos(data_clean, '일')

# Selecting new features including the sin/cos transformed date features
features_extended = features + ['년_sin', '년_cos', '월_sin', '월_cos', '일_sin', '일_cos']

X_extended = data_clean[features_extended]

# Splitting the dataset into training and testing set with extended features
X_train_ext, X_test_ext, y_train, y_test = train_test_split(X_extended, y, test_size=0.2, random_state=42)

# Define model with the extended features
model_ext = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100, random_state=42))

# Train the model with extended features
model_ext.fit(X_train_ext, y_train)

# Predict on the testing set with extended features
y_pred_ext = model_ext.predict(X_test_ext)

# Calculate the mean squared error with extended features
mse_ext = mean_squared_error(y_test, y_pred_ext)

mse_ext


/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['년_sin'], data_clean['년_cos'] = date_to_sin_cos(data_clean, '년')
/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['년_sin'], data_clean['년_cos'] = date_to_sin_cos(data_clean, '년')
/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:12: SettingWithCopyWarning

0.2544575758140972

In [6]:
import numpy as np

# Function to convert dates to sin/cos values
def date_to_sin_cos(df, column_name):
    max_value = df[column_name].max()
    sin_col = np.sin(2 * np.pi * df[column_name] / max_value)
    cos_col = np.cos(2 * np.pi * df[column_name] / max_value)
    return sin_col, cos_col

# Applying the conversion
data_clean['년_sin'], data_clean['년_cos'] = date_to_sin_cos(data_clean, '년')
data_clean['월_sin'], data_clean['월_cos'] = date_to_sin_cos(data_clean, '월')
data_clean['일_sin'], data_clean['일_cos'] = date_to_sin_cos(data_clean, '일')

# Selecting new features including the sin/cos transformed date features
features_extended = features + ['년_sin', '년_cos', '월_sin', '월_cos', '일_sin', '일_cos']

X_extended = data_clean[features_extended]

# Splitting the dataset into training and testing set with extended features
X_train_ext, X_test_ext, y_train, y_test = train_test_split(X_extended, y, test_size=0.2, random_state=42)

# Define model with the extended features
model_ext = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100, random_state=42))

# Train the model with extended features
model_ext.fit(X_train_ext, y_train)

# Predict on the testing set with extended features
y_pred_ext = model_ext.predict(X_test_ext)

# Calculate the mean squared error with extended features
mse_ext = mean_squared_error(y_test, y_pred_ext)

mse_ext


/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['년_sin'], data_clean['년_cos'] = date_to_sin_cos(data_clean, '년')
/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['월_sin'], data_clean['월_cos'] = date_to_sin_cos(data_clean, '월')
/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/3704540372.py:13: SettingWithCopyWarning

0.2544575758140972

### 계절성 추가


In [7]:
# Function to add seasonality feature based on month
def add_seasonality(df):
    seasons = [(1, 'Winter'), (2, 'Winter'), (3, 'Spring'), (4, 'Spring'), (5, 'Spring'), 
               (6, 'Summer'), (7, 'Summer'), (8, 'Summer'), (9, 'Autumn'), (10, 'Autumn'), 
               (11, 'Autumn'), (12, 'Winter')]
    month_to_season = {month: season for month, season in seasons}
    df['Season'] = df['월'].map(month_to_season)
    return df

# Adding seasonality to the clean dataset
data_seasonal = add_seasonality(data_clean)

# Updating features list to include the new 'Season' feature
features_seasonal = features_extended + ['Season']

# Applying OneHotEncoder to the 'Season' feature along with other categorical features
preprocessor_seasonal = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(), categorical_features + ['Season'])
    ]
)

# Updating the dataset for model training
X_seasonal = data_seasonal[features_seasonal]
y_seasonal = data_seasonal[target]

# Splitting the dataset into training and testing set with seasonality feature
X_train_sea, X_test_sea, y_train_sea, y_test_sea = train_test_split(X_seasonal, y_seasonal, test_size=0.2, random_state=42)

# Define model with the seasonal features
model_sea = make_pipeline(preprocessor_seasonal, RandomForestRegressor(n_estimators=100, random_state=42))

# Train the model with seasonal features
model_sea.fit(X_train_sea, y_train_sea)

# Predict on the testing set with seasonal features
y_pred_sea = model_sea.predict(X_test_sea)

# Calculate the mean squared error with seasonal features
mse_sea = mean_squared_error(y_test_sea, y_pred_sea)

mse_sea


/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/1983168076.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season'] = df['월'].map(month_to_season)


0.23283745101342007

In [9]:
# Adding seasonality to '민형기' data
min_hyung_ki_data_seasonal = add_seasonality(min_hyung_ki_data)

# Preparing '민형기' data with seasonal features for prediction
min_hyung_ki_features_sea = min_hyung_ki_data_seasonal[features_seasonal].iloc[0].to_frame().T  # Using the first row as representative

# Predict '민형기's viewership rating with seasonal features
min_hyung_ki_predicted_rating_sea = model_sea.predict(min_hyung_ki_features_sea)

min_hyung_ki_predicted_rating_sea


/var/folders/3d/nr_splbx619_jz_ryc6wg6740000gn/T/ipykernel_2019/1983168076.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season'] = df['월'].map(month_to_season)


KeyError: "['년_sin', '년_cos', '월_sin', '월_cos', '일_sin', '일_cos'] not in index"